# Databricks Concurrency Benchmark

This notebook demonstrates a comprehensive concurrency benchmarking workflow using Databricks SQL and Delta tables. The benchmark involves loading large datasets from external storage, transforming data using VARIANT columns, and creating structured Delta tables for performance testing.

## Overview
- **Dataset**: Line items data (~60M records)
- **Storage**: Azure Data Lake Storage Gen2
- **Format**: Parquet to Delta transformation
- **Focus**: Concurrency and performance benchmarking

## 1. Setup Environment and Configuration

First, we'll import the necessary libraries and configure our environment for connecting to Databricks.

## 2. Explore External Storage Locations

Let's start by examining the external storage locations and exploring the source data structure.

In [ ]:
%sql
-- Show available external locations
SHOW EXTERNAL LOCATIONS;

In [ ]:
%sql
-- List files in the source location to understand data structure
LIST 'abfss://demo@dbxdl.dfs.core.windows.net/lineitems/' WITH (CREDENTIAL `dbxdl-storage-account-creds`);

## 3. Create and Configure Catalog Structure

Now we'll set up the catalog and schema structure for our benchmark test.

In [ ]:
%sql
-- Show existing catalogs
SHOW CATALOGS;

In [ ]:
%sql
-- Create the catalog for concurrency testing
DROP CATALOG IF EXISTS 04_CONCURRENCY CASCADE;
CREATE CATALOG IF NOT EXISTS 04_CONCURRENCY;

In [ ]:
%sql
-- Set the catalog and schema context
USE CATALOG 04_CONCURRENCY;
USE SCHEMA default;

### Optional: Custom Schema Location
If you need the schema in a different location than the default catalog location, you can uncomment and run the following command:

In [ ]:
%sql
-- Uncomment if you need a custom schema location
-- CREATE SCHEMA IF NOT EXISTS BRONZE MANAGED LOCATION "abfss://delta@dbxdl.dfs.core.windows.net/default/";

## 4. Examine Source Parquet Files

Let's read a few sample records from the source Parquet files to understand the data structure.

In [ ]:
%sql
-- Reading few records from the source Parquet files
SELECT * FROM PARQUET.`abfss://warehouse@dbxdl.dfs.core.windows.net/lineitems/*` LIMIT 5;

## 5. Create Raw Delta Table with VARIANT Column

We'll create a raw table using the VARIANT data type to store the semi-structured data from Parquet files.

In [ ]:
%sql
-- Creating RAW_LINEITEMS_WAREHOUSE table to land the data
CREATE OR REPLACE TABLE 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE (
  V VARIANT
) USING DELTA;

In [ ]:
%sql
-- Verify the table is empty initially
SELECT * FROM 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE LIMIT 10;

## 6. Load Data Using COPY INTO

⚠️ **Important Performance Note**: 
- COPY INTO must list the source path every run to decide what's new
- Listing millions of files on ADLS Gen2 becomes slow and expensive (many list calls)
- Consider Auto Loader's clean source options to DELETE/MOVE files after ~30 days

In [ ]:
%sql
-- Copying the data from Parquet files into the Delta table
COPY INTO 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE
FROM (
  SELECT parse_json(to_json(struct(*))) AS V 
  FROM 'abfss://warehouse@dbxdl.dfs.core.windows.net/lineitems/*'
)
FILEFORMAT = PARQUET
FORMAT_OPTIONS ('singleVariantColumn' = 'true');

## 7. Validate Data Load and Count Records

Let's verify the data was loaded successfully and count the total records.

In [ ]:
%sql
-- Count total records (should be around 59,986,052)
SELECT COUNT(*) FROM 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE;

In [ ]:
%sql
-- View sample records to verify structure
SELECT * FROM 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE LIMIT 10;

## 8. Extract Fields from VARIANT Data

Let's extract and examine specific fields from the VARIANT column using the `variant_get()` function.

In [ ]:
%sql
-- Select 5 random values to examine the data structure
SELECT
  variant_get(V, '$.L_ORDERKEY') AS order_key,
  variant_get(V, '$.L_PARTKEY') AS part_key,
  variant_get(V, '$.L_SUPPKEY') AS supp_key,
  variant_get(V, '$.L_LINENUMBER') AS line_number,
  variant_get(V, '$.L_QUANTITY') AS quantity,
  variant_get(V, '$.L_EXTENDEDPRICE') AS extended_price,
  variant_get(V, '$.L_DISCOUNT') AS discount,
  variant_get(V, '$.L_TAX') AS tax,
  variant_get(V, '$.L_RETURNFLAG') AS return_flag,
  variant_get(V, '$.L_LINESTATUS') AS line_status,
  variant_get(V, '$.L_SHIPDATE') AS ship_date,
  variant_get(V, '$.L_COMMITDATE') AS commit_date,
  variant_get(V, '$.L_RECEIPTDATE') AS receipt_date,
  variant_get(V, '$.L_SHIPINSTRUCT') AS ship_instruct,
  variant_get(V, '$.L_SHIPMODE') AS ship_mode,
  variant_get(V, '$.L_COMMENT') AS comment
FROM 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE
ORDER BY RAND() LIMIT 5;

In [ ]:
%sql
-- Query records with specific ORDERKEY values (requires casting)
SELECT
  variant_get(V, '$.L_ORDERKEY') AS order_key,
  variant_get(V, '$.L_PARTKEY') AS part_key,
  variant_get(V, '$.L_SUPPKEY') AS supp_key,
  variant_get(V, '$.L_LINENUMBER') AS line_number,
  variant_get(V, '$.L_QUANTITY') AS quantity,
  variant_get(V, '$.L_EXTENDEDPRICE') AS extended_price,
  variant_get(V, '$.L_DISCOUNT') AS discount,
  variant_get(V, '$.L_TAX') AS tax,
  variant_get(V, '$.L_RETURNFLAG') AS return_flag,
  variant_get(V, '$.L_LINESTATUS') AS line_status,
  variant_get(V, '$.L_SHIPDATE') AS ship_date,
  variant_get(V, '$.L_COMMITDATE') AS commit_date,
  variant_get(V, '$.L_RECEIPTDATE') AS receipt_date,
  variant_get(V, '$.L_SHIPINSTRUCT') AS ship_instruct,
  variant_get(V, '$.L_SHIPMODE') AS ship_mode,
  variant_get(V, '$.L_COMMENT') AS comment
FROM 04_CONCURRENCY.default.RAW_LINEITEMS_WAREHOUSE
WHERE CAST(variant_get(V, '$.L_ORDERKEY') AS STRING) IN (
  '30370724',
  '43675749',
  '46386755',
  '39896960',
  '51780611'
);

## 9. Create Structured Delta Table

We'll now transform the semi-structured variant data into a fully typed Delta table for improved query performance.

In [ ]:
%sql
-- Create the final structured Delta table
CREATE OR REPLACE TABLE 04_CONCURRENCY.default.LINEITEMS_WAREHOUSE
USING DELTA
AS
SELECT
  CAST(variant_get(V, '$.L_ORDERKEY') AS VARCHAR(15)) AS L_ORDERKEY,
  CAST(variant_get(V, '$.L_PARTKEY') AS VARCHAR(15)) AS L_PARTKEY,
  CAST(variant_get(V, '$.L_SUPPKEY') AS VARCHAR(15)) AS L_SUPPKEY,
  CAST(variant_get(V, '$.L_LINENUMBER') AS INT) AS L_LINENUMBER,
  CAST(variant_get(V, '$.L_QUANTITY') AS FLOAT) AS L_QUANTITY,
  CAST(variant_get(V, '$.L_EXTENDEDPRICE') AS FLOAT) AS L_EXTENDEDPRICE,
  CAST(variant_get(V, '$.L_DISCOUNT') AS FLOAT) AS L_DISCOUNT,
  CAST(variant_get(V, '$.L_TAX') AS FLOAT) AS L_TAX,
  CAST(variant_get(V, '$.L_RETURNFLAG') AS VARCHAR(30)) AS L_RETURNFLAG,
  CAST(variant_get(V, '$.L_LINESTATUS') AS VARCHAR(30)) AS L_LINESTATUS,
  CAST(variant_get(V, '$.L_SHIPDATE') AS DATE) AS L_SHIPDATE,
  CAST(variant_get(V, '$.L_COMMITDATE') AS DATE) AS L_COMMITDATE,
  CAST(variant_get(V, '$.L_RECEIPTDATE') AS DATE) AS L_RECEIPTDATE,
  CAST(variant_get(V, '$.L_SHIPINSTRUCT') AS VARCHAR(30)) AS L_SHIPINSTRUCT,
  CAST(variant_get(V, '$.L_SHIPMODE') AS VARCHAR(30)) AS L_SHIPMODE,
  CAST(variant_get(V, '$.L_COMMENT') AS VARCHAR(100)) AS L_COMMENT
FROM 04_CONCURRENCY.default.raw_lineitems_warehouse;

In [ ]:
%sql
-- Validate the final table
SELECT * FROM 04_CONCURRENCY.default.lineitems_warehouse LIMIT 10;

## 10. Performance Testing and Validation

You can now run concurrency tests against the structured Delta table. Suggested queries:
- Random sampling
- Aggregations on quantity and price
- Filtering by date ranges
- Joining with dimension tables (if available)

Below are placeholders for performance-focused queries.

In [ ]:
%sql
-- Count records in the final table
SELECT COUNT(*) FROM 04_CONCURRENCY.default.lineitems_warehouse;

In [ ]:
%sql
-- Example aggregation for performance testing
SELECT L_SHIPMODE, COUNT(*) AS cnt, SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS revenue
FROM 04_CONCURRENCY.default.lineitems_warehouse
GROUP BY L_SHIPMODE
ORDER BY revenue DESC;

## 11. Cleanup Operations

Use the following commands to clean up all objects created during the benchmark.

In [ ]:
%sql
-- CLEAN UP (uncomment as needed)
-- USE CATALOG 04_CONCURRENCY;
-- USE SCHEMA default;

-- DROP TABLE IF EXISTS 04_CONCURRENCY.default.lineitems;
-- DROP TABLE IF EXISTS 04_CONCURRENCY.default.lineitems_warehouse;
-- DROP TABLE IF EXISTS 04_CONCURRENCY.default.raw_lineitems;
-- DROP TABLE IF EXISTS 04_CONCURRENCY.default.raw_lineitems_warehouse;
-- DROP TABLE IF EXISTS 04_CONCURRENCY.default.raw_lineitems_dlt;

-- DROP SCHEMA IF EXISTS 04_CONCURRENCY.default CASCADE;
-- DROP SCHEMA IF EXISTS 04_CONCURRENCY.silver CASCADE;
-- DROP SCHEMA IF EXISTS 04_CONCURRENCY.default CASCADE;